In [1]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

In [2]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

在底层，NumPy 调用库（例如 OpenBLAS）和它自己在低级 C 语言中的一些实现来执行这些计算。
从张量程序抽象的角度来看，我们想彻底理解这些数组计算的背后的细节。具体来说，我们想问：实现相应计算的可能方式是什么？

为了说明底层细节，我们将在 NumPy API 的一个受限子集中编写示例 —— 我们称之为 低级 NumPy。它使用以下的约定：
* 我们将在必要时使用循环而不是数组函数来展示可能的循环计算。
* 如果可能，我们总是通过 numpy.empty 显式地分配数组并传递它们。

需要注意的是，这不是人们通常编写 NumPy 程序的方式。不过，它们仍然与幕后发生的事情非常相似 —— 大多数现实世界的部署解决方案都将分配与计算分开处理。特定的库使用不同形式的循环和算术计算来执行计算。当然首先它们是使用诸如 C 之类的低级语言实现的。

In [3]:
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype="float32")
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)

上面的程序是实现 mm_relu 操作的一种方式。该程序包含两个阶段：首先我们分配一个中间存储，将矩阵乘法的结果存储在那里。然后我们在第二个 for 循环序列中计算 ReLU。你可能会注意到，这肯定不是实现 mm_relu 的唯一方法，当然这可能也不是你想到的第一件事。

无论如何，这确实是实现 mm_relu 的方式之一。我们可以通过将我们的结果与使用数组计算的原始结果进行比较来验证代码的正确性。我们将在本节的后面部分回到这里，并重新讨论其他可能的方法。

In [4]:
c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)

上面的示例代码展示了我们如何在幕后实现 mm_relu。当然，由于 Python 解释器，代码本身会运行得很慢。尽管如此，示例 NumPy 代码包含我们将在这些计算的实际实现中使用的所有可能元素。
* 多维缓冲区（数组）。
* 在数组维度上的循环。
* 在循环下执行的计算语句。